# Поиск токсичных комментариев

### Задача

---

Интернет-магазин запускает сервис, в которм пользователи могут редактировать и дополнять описания товаров. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные.

*Условие*: метрика качества *F1* должна быть не меньше 0.75.

### Данные

---

Набор данных с разметкой о токсичности правок:
- `text` - текст комментария
- `toxic` - является ли токсичным

### План проекта:

---
1. Подготовка данных.
    - Знакомство с данными. Выделение обучающей и тестовой подвыборок. Получение эмбеддингов.
2. Обучение моделей.
    - Логистическая регрессия, случайный лес, LightGBM, CatBoost. Тестирование лучшей модели.
3. Вывод.

---


## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
RANDOM_STATE = 42

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce GTX 1060


### Чтение данных

In [4]:
data = pd.read_csv("../datasets/toxic_comments.csv")

Посмотрим на общую информацию о датасете.

In [5]:
def data_info(data):
    display(data.head())
    print()
    print(data.info())
    print(f"\nЯвных дубликатов: {data.duplicated().sum()}")
    print("\nПодсчет классов:")
    print(data['toxic'].value_counts())

In [6]:
data_info(data)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None

Явных дубликатов: 0

Подсчет классов:
toxic
0    143106
1     16186
Name: count, dtype: int64


**Промежуточный вывод:**

Датасет содержит 159 292 текстовых комментария с разметкой токсичности `toxic`. Решается задача бинарной классификации. Явных дубликатов нет. Наблюдается сильный дисбаланс классов: токсичных комментариев всего около 10%.

Для обучения моделей данные нужно подготовить. Что нужно сделать:

- избавиться от лишних столбцов;
- учесть дисбаланс классов;
- векторизовать тексты.

### Подготовка данных

Удалим из датасета ненужный столбец `Unnamed: 0`, так как он дублирует индексы.

In [7]:
data = data.drop('Unnamed: 0', axis=1)

Датасет довольно большой, и получение эмбеддингов с помощью BERT займет довольно много времени. Поэтому ограничимся лишь частью данных. Возьмем выборку из 1000 объектов и проверим, что соотношение классов примерно то же.

In [8]:
comments = data.sample(n=1000, random_state=RANDOM_STATE).reset_index(drop=True)
comments['toxic'].value_counts()

toxic
0    894
1    106
Name: count, dtype: int64

Объектов положительного класса также ~10%.

Разделим данные на обучающую и тестовую выборки в соотношении 8:2 с учетом стратификации.

In [9]:
comments_train, comments_test, y_train, y_test = train_test_split(
    comments['text'], comments['toxic'], test_size=0.2, stratify=comments['toxic'], random_state=RANDOM_STATE)

print('Размеры выборок:')
print(f"train - {len(comments_train)} - {len(comments_train)/len(comments['text']):.0%}")
print(f"test - {len(comments_test)} - {len(comments_test)/len(comments['text']):.0%}")

Размеры выборок:
train - 800 - 80%
test - 200 - 20%


### Эмбеддинги

Векторизуем текст в эмбеддинги с помощью предобученной модели [BERT](https://huggingface.co/unitary/toxic-bert/tree/main). Напишем для этого функцию.

In [10]:
def get_features(comments):
    ## Токенизируем тексты комментариев
    tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert', max_length=512)
    tokenized = comments.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    
    ## Приведем все векторы токенов к одному размеру
    max_len = 512
    padded = np.array([i + [0]*(max_len - len(i)) if len(i)<512  else i[:512]  for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)

    print(f"tokenized shape: {tokenized.shape}")
    print(f"padded shape: {padded.shape}")
    print(f"attention_mask shape: {attention_mask.shape}")
    
    ## Инициализируем модель и запустим ее на cuda
    config = config = transformers.BertConfig.from_pretrained('unitary/toxic-bert')
    model = transformers.BertModel.from_pretrained('unitary/toxic-bert', config=config).cuda()
    
    ## Будем создавать эмбеддинги батчами по 20 текстов. Размер батча определен эмпирически.
    batch_size = 20
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.cuda.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.cuda.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        
    return np.concatenate(embeddings)

In [11]:
# torch.cuda.empty_cache()
# print(torch.cuda.memory_reserved())

Получим векторное представление текстов отдельно для обучающей и тестовой выборок, чтобы не возникло эффекта подглядывания.

In [12]:
X_train = get_features(comments_train)
print(f"features shape: {X_train.shape}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1279 > 512). Running this sequence through the model will result in indexing errors


tokenized shape: (800,)
padded shape: (800, 512)
attention_mask shape: (800, 512)


  0%|          | 0/40 [00:00<?, ?it/s]

features shape: (800, 768)


In [13]:
X_test = get_features(comments_test)
print(f"features shape: {X_test.shape}")

Token indices sequence length is longer than the specified maximum sequence length for this model (572 > 512). Running this sequence through the model will result in indexing errors


tokenized shape: (200,)
padded shape: (200, 512)
attention_mask shape: (200, 512)


  0%|          | 0/10 [00:00<?, ?it/s]

features shape: (200, 768)


## Обучение моделей

Обучим несколько моделей: логистическую регрессию, случайный лес, LightGBM, CatBoost. Дисбаланс классов учтем с помощью весов.

**1)** LogisticRegression

In [14]:
%%time
logreg = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced')
print('F1:', cross_val_score(logreg, X_train, y_train, scoring='f1', cv=4).mean())

logreg.fit(X_train, y_train)

F1: 0.959830866807611
CPU times: total: 1.7 s
Wall time: 295 ms


LogisticRegression(class_weight='balanced', random_state=42)

**2)** RandomForestClassifier

In [15]:
def fit_model(estimator, param_grid, X_train, y_train):
    model = GridSearchCV(estimator=estimator, 
                            param_grid=param_grid, 
                            cv=3,
                            scoring='f1')

    model.fit(X_train, y_train)

    best_index = model.best_index_
    best_rmse = round(model.cv_results_['mean_test_score'][best_index], 4)

    print(f"Best F1: {best_rmse}")
    print(f"Best params: {model.best_params_}")

    return model.best_estimator_

In [16]:
%%time
rf_estimator = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced')

rf_param_grid =  {
    'n_estimators': list(range(60, 121, 30)),
    'max_depth': list(range(2, 13, 5)),
}

rf_best_model = fit_model(
    estimator=rf_estimator,
    param_grid=rf_param_grid,
    X_train=X_train,
    y_train=y_train
)

Best F1: 0.96
Best params: {'max_depth': 2, 'n_estimators': 90}
CPU times: total: 11.6 s
Wall time: 10.9 s


**3)** LGBMClassifier

In [17]:
%%time
lgbm_estimator = LGBMClassifier(random_state=RANDOM_STATE, class_weight='balanced', verbose=-1)

lgbm_param_grid = {
    "n_estimators": range(25, 101, 25), 
    "max_depth": range(5, 16, 5),
    'learning_rate': [0.15, 0.2, 0.25]
}

lgbm_best_model = fit_model(
    estimator=lgbm_estimator,
    param_grid=lgbm_param_grid,
    X_train=X_train,
    y_train=y_train
)

Best F1: 0.9772
Best params: {'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 25}
CPU times: total: 5min 19s
Wall time: 53.7 s


**4)** CatBoostClassifier

In [18]:
%%time
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

catboost_estimator = CatBoostClassifier(random_state=RANDOM_STATE, verbose=False, class_weights=class_weights)


catboost_param_grid = {
    "iterations": range(100, 201, 50),
    "learning_rate": [0.1, 0.15, 0.2],
    "depth": range(2, 7, 2)
}

catboost_best_model = fit_model(
    estimator=catboost_estimator,
    param_grid=catboost_param_grid,
    X_train=X_train,
    y_train=y_train
)

Best F1: 0.955
Best params: {'depth': 6, 'iterations': 100, 'learning_rate': 0.1}
CPU times: total: 1h 37min 10s
Wall time: 9min 42s


В качестве лучшей модели выберем LGBMClassifier, так как метрика F1 у нее лучше остальных.

### Тестирование

Проверим качесто выбранной модели на тестовой выборке.

In [19]:
print(f'F1 на тестовой выборке: {f1_score(y_test, lgbm_best_model.predict(X_test)):.4}')

F1 на тестовой выборке: 0.9545


Качество лучше требуемого F1 = 0.75.

## Вывод

Целью проекта была разработка модели для определения токсичности комментариев.

В нашем распоряжении был датасет с разметкой о токсичности, который содержал 159 292 текстовых комментария. С целью ускорения расчетов было решено взять для обучения модели только часть данных - 1000 комментариев.

Тексты были переведены в эмбеддинги с помощью модели BERT.

На эмбеддингах были обучены четыре модели: логистическая регрессия, случайный лес, LightGBM и CatBoost. Среди этих моделей по качеству лучшей оказалась модель **LGBMClassifier** с параметрами: learning_rate=0.2, max_depth=5, n_estimators=50. На тестовой выборке лучшая модель показала **F1 = 0.95**, что лучше требуемого F1 = 0.75.